In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
cd "/content/gdrive/My Drive/cse519_final_proj/dataset"

/content/gdrive/My Drive/cse519_final_proj/dataset


In [ ]:
pip install wget

In [ ]:
cd ../WorkingD/data/pre_process_data

/content/gdrive/My Drive/cse519_final_proj/WorkingD/data/pre_process_data


In [ ]:
import os

files_list = []

for root, dirs, files in os.walk("./01"):
    for file in files:
        if file.endswith(".json"):
             files_list.append(str(os.path.join(root, file)))

In [ ]:
import json
import pandas as pd

geos = pd.DataFrame([])

geo_cnt = 0
place_cnt = 0
user_loc_cnt = 0

for file_name in files_list:
    temp = pd.read_json(file_name, orient='records',convert_dates=False)
    geos = pd.concat([geos,temp], ignore_index=True)

In [ ]:
def fill_coordinates(row):
    try:
        coordinates = row['place']['bounding_box']['coordinates']
        return (coordinates[0][0][1], coordinates[0][0][0])
    except:
        return None

geos['coordinate'] = geos.apply(fill_coordinates, axis = 1)

In [ ]:
geos.dropna(inplace=True)
geos.drop(columns=['place'], inplace=True)

In [ ]:
pip install reverse_geocoder pytz timezonefinder

     |████████████████████████████████| 39.2MB 89kB/s 


In [ ]:
coords = geos['coordinate'].tolist()
coords = tuple(coords)

import reverse_geocoder as rg
results = rg.search(coords)

country = []
state = []

for rec in results:
    country.append(rec['cc'])
    state.append(rec['admin1'])

country = pd.Series(country)
state = pd.Series(state)

In [ ]:
geos['country'] = country
geos['state'] = state
geos.dropna(inplace=True)

In [ ]:
from timezonefinder import TimezoneFinder
import datetime
import pytz

tf = TimezoneFinder()

def get_localtime_in_seconds(rows):
    coor = rows['coordinate']
    lati = coor[0]
    logi = coor[1]
    zone_str = tf.timezone_at(lng = logi, lat = lati)
    if zone_str == None:
        return None
    d = datetime.datetime.fromtimestamp(rows['timestamp_ms']/1000, tz=pytz.timezone(zone_str))
    h, m, s = d.hour, d.minute, d.second
    return h*3600+m*60+s

geos.apply(get_localtime_in_seconds, axis=1)

0          18604.0
1          51006.0
2           7808.0
3          58210.0
4           4210.0
            ...   
1222282     2445.0
1222283     6047.0
1222284    16981.0
1222285     2583.0
1222286    63785.0
Length: 1222273, dtype: float64

In [ ]:
import matplotlib.pyplot as plt

plt

In [ ]:
def get_location(row):
    try:
        loc = row['user']['location']
    except:
        return 'none'
    return loc
loc = df.apply(get_location, axis=1)
print(loc.unique())

['さくらんだいすきにんげん' 'INDONESIA' 'none' ... '살려주세오' 'Morioh'
 'langitlumpaimpyerno']


In [ ]:
df.iloc[0]['user']

{'contributors_enabled': False,
 'created_at': 'Mon Aug 26 23:52:03 +0000 2019',
 'default_profile': True,
 'default_profile_image': False,
 'description': '猫とゲームとアニメが好きな気分屋で超絶人見知りな人間ですがよろしくお願いしますですます。 ※冬は基本お布団かこたつの中に生息しています。¦無言フォロー失礼します¦ ふぉろば100%¦さくらんだいすき▷▶▷@LGS_PROCLASS',
 'favourites_count': 62466,
 'follow_request_sent': None,
 'followers_count': 5053,
 'following': None,
 'friends_count': 5233,
 'geo_enabled': False,
 'id': 1166136217785655296,
 'id_str': '1166136217785655296',
 'is_translator': False,
 'lang': None,
 'listed_count': 33,
 'location': 'さくらんだいすきにんげん',
 'name': '愛美🐳🌟',
 'notifications': None,
 'profile_background_color': 'F5F8FA',
 'profile_background_image_url': '',
 'profile_background_image_url_https': '',
 'profile_background_tile': False,
 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/1166136217785655296/1576381562',
 'profile_image_url': 'http://pbs.twimg.com/profile_images/1207116713411502080/eLG7jOI2_normal.jpg',
 'profile_image_url_https': 'ht

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import io
import json
import bz2
import numpy as np
import pandas as pd
import multiprocessing
from joblib import Parallel, delayed

# the date start in year-month "/day/hour"
# 
# dir = './24/19/'
# file path like 
month = 5
day = 1
dir_list =[] 
f_name_list = []

# list of hours 
for hour in range(0,24):
    dir_list.append("./data/%02d/%02d/%02d"%(month,day,hour))
    f_name_list.append("%02d_%02d_%02d"%(month,day,hour))

intput_list = zip(dir_list, f_name_list)

def loc_aviable_filter(path, fname):
    loc_avalibel_tweets = []
    print(f"process the file {fname}")
    # get the json files in hour
    files = os.listdir(path)
    
    try:
        files = os.listdir(path)
    except:
        return 
    
    if not files:
        return 
    
    # parse the json file
    for file in files:
        sample_file = path + '/'+file
        
        with bz2.open(sample_file, "rt") as bzinput:
            for i, line in enumerate(bzinput):
                tweets = json.loads(line)

                if tweets.get('place'):
                    loc_avalibel_tweets.append({'user_id': tweets['user']['id'], 'place': tweets['place'], 'timestamp_ms': tweets['timestamp_ms']})
    
    df = pd.DataFrame(data=loc_avalibel_tweets)
    # output to the json file
    df.to_json (f'./data/pre_process_data/{fname}.json',orient='records')
    print(f"finish the file {fname}")

Parallel(n_jobs=14)(delayed(loc_aviable_filter)(path,fname) for path, fname in intput_list)

In [ ]:
pip install reverse_geocoder

     |████████████████████████████████| 2.3MB 2.8MB/s 
  Created wheel for reverse-geocoder: filename=reverse_geocoder-1.5.1-cp36-none-any.whl size=2268090 sha256=a4b859846165b5746939641737507c942bf8eea3f2bebca95c0588a6604501dd
  Stored in directory: /root/.cache/pip/wheels/47/05/50/b1350ff094ef91e082665b4a2f9ca551f8acea4aa55d796b26
Successfully built reverse-geocoder


In [ ]:
import reverse_geocoder as rg

coordinates = (40.916299, -73.126859)

results = rg.search(coordinates) # default mode = 2

print(results) 

[OrderedDict([('lat', '40.92565'), ('lon', '-73.14094'), ('name', 'Stony Brook'), ('admin1', 'New York'), ('admin2', 'Suffolk County'), ('cc', 'US')])]
